# NLP - Part 2 - Chatbot (GUI)

In [1]:
# Importing dependencies

import numpy as np

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
import re
from sklearn.preprocessing import LabelEncoder

import keras, tensorflow
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.optimizers import SGD

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

import json
import random
import pickle

from nltk.stem import WordNetLemmatizer 
import string

from keras.models import load_model

## Import data

In [2]:
# import corpus file
with open('GL Bot.json') as file:
    corpus = json.load(file)

# Display corpus file
print(corpus)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of p

In [3]:
# List initialization
corp1 = corpus["intents"]
corp2 = [
    {'tag': 'options', 
     'patterns': ["How can you help me", "What can you do", "What help you provide", "What do you know"],
     'responses': ["I can guide you with machine learning and artificial learning concepts", "I can assist you with your program"], 
     'context_set': ''
    },
    {'tag': 'SL', 
     'patterns': ['bias-variance tradeoff', 'support vector machines', 'normalization', 'Can you help me understand svm', 
                  'could you please explain me how machine learning works', 'K nearest neighbor algorithm'
                  'Could me help me understand naive bayes', 'decision trees', 'labeled data', 'supervised', 'Do you know KNN',
                  'Can you help me with KNN algorithm', 'Can you help me with supervised ML?', 'Can you explain KNN'], 
     'responses': ['Link: https://en.wikipedia.org/wiki/Supervised_learning'], 
     'context_set': ''
    },
    {'tag': 'USL', 
     'patterns': ['PCA', 'Pricipal component analysis', 'K means clustering', 'dimensionality reduction', 'unlabeled data'
                  'clustering', 'K nearest neighbor algorithm', 'clustering', 'association', 'hierarchical clustering'
                  'Singular value decomposition', 'Do you know PCA'], 
     'responses': ['Link: https://en.wikipedia.org/wiki/Unsupervised_learning'], 
     'context_set': ''
    },
    {'tag': 'NN', 
     'patterns': ['RelU', 'sigmoid function', 'tanh', 'optimizer', 'neural networks', 'Can you help me with deep learning',
                  'ANN', 'CNN', 'Transfer learning'],
     'responses': ['Link: https://en.wikipedia.org/wiki/Artificial_neural_network'],
     'context_set': ''
    },
    {'tag': 'Bot', 
     'patterns': ['May I know who you are', "whats you're name"],
     'responses': ["I'm Proton, your virtual assistant and I'm 24x7 available"],
     'context_set': ''
    },
    {'tag': 'no answer', 
     'patterns': [],
     'responses': ["Sorry, I can't understand you", "Please give me more info", "Unable to understand you, Can you provide info"], 
     'context_set': ''
    },
    {'tag': 'Intro', 
     'patterns': ['How do you do', 'Hi.. Can you help me with my issue', 'Are you there', "what's up", "Howdy", 
                  'Could you please help me', 'I need you to listen to me', 'I belong to a new batch of', 
                  'Artificial Intelligence class', 'AI & ML batch', 'Machine Learning batch', 'My Program Manager is'], 
     'responses': ['Hi there, how can I help you', 'Good to see you again', 'Hello thanks for asking', 
                   'Hello, How may I help you'], 
     'context_set': ''
    },
    {'tag': 'Exit',
     'patterns': ["that was very helpful", 'thanks again', 'awesome, thanks', 'thanks for helping me out', 
                  'will catch you later', "i'll take a leave", 'have a Great day', 'you helped a lot', 
                  'you are a great learning buddy', 'bye', 'good bye'],
     'responses': ['Anytime!!', "Happy to help!", "My pleasure!", "Glad I could assist you"],
     'context_set': ''
    },
    {'tag': 'Ticket', 
     'patterns': ['my issue is not solved', 'you answer did not help', 'I did not get the answer', 'solution was not good', 
                  'you did not help', 'I want to talk to a my PM', 'connect me to the program manager', 
                  'I want to raise a ticket', 'no'],
     'responses': ['Transferring the request to your PM', 'Please wait while I transfer your request to PM'], 
     'context_set': ''
    }
    ]

In [4]:
# Iterating and using extend to convert
for elm2 in corp2:
  
    for elm1 in corp1:
        if elm2['tag'] == elm1['tag']:
            elm1['responses'].extend(elm2['responses'])
            break
    else:
        corp1.append(elm2)
        
# printing 
# print(corp1)

In [5]:
corpus["intents"] = corp1

In [6]:
print(corpus['intents'])

[{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?', 'Hi there, how can I help you', 'Good to see you again', 'Hello thanks for asking', 'Hello, How may I help you'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye', 'Anytime!!', 'Happy to help!', 'My pleasure!', 'Glad I could assist you'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to und

## Data pre-processing

In [7]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

Tokenizing is the most basic and first thing you can do on text data. Tokenizing is the process of breaking the whole text into small parts like words.

In [8]:
for intent in corpus['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

We will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [9]:
# lemmatize, lower each word and remove duplicates
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print("There are {} documents.".format(len(documents)))
print('')

# classes = intents
print (len(classes), "classes:", classes)
print('')

# words = all words, vocabulary
print (len(words), "unique lemmatized words:", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

There are 142 documents.

10 classes: ['Bot', 'Exit', 'Intro', 'NN', 'Olympus', 'Profane', 'SL', 'Ticket', 'USL', 'options']

180 unique lemmatized words: ['a', 'able', 'access', 'activation', 'ada', 'adam', 'aifl', 'aiml', 'algorithm', 'am', 'an', 'analysis', 'ann', 'anyone', 'are', 'artificial', 'association', 'backward', 'bad', 'bagging', 'batch', 'bayes', 'belong', 'best', 'blended', 'bloody', 'boosting', 'bot', 'buddy', 'can', 'classification', 'clustering', 'clusteringsingular', 'component', 'contact', 'create', 'cross', 'cya', 'dataclustering', 'day', 'decomposition', 'deep', 'did', 'diffult', 'dimensionality', 'do', 'ensemble', 'epoch', 'explain', 'first', 'for', 'forest', 'forward', 'from', 'function', 'good', 'goodbye', 'gradient', 'great', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'hierarchical', 'hour', 'how', 'hyper', 'i', 'imputer', 'in', 'intelligence', 'is', 'jerk', 'joke', 'k', 'knn', 'know', 'later', 'layer', 'learner', 'learning', 'lea

## Model building

Now we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [10]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


Now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, let us save the model as ‘chatbot_model.h5’.

In [11]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('proton_model.h5', hist)

print("Model created.")

Epoch 1/200
29/29 [==============================] - 0s 641us/step - loss: 2.2861 - accuracy: 0.1169
Epoch 2/200
29/29 [==============================] - 0s 534us/step - loss: 2.1267 - accuracy: 0.2526
Epoch 3/200
29/29 [==============================] - 0s 499us/step - loss: 2.0256 - accuracy: 0.2681
Epoch 4/200
29/29 [==============================] - 0s 499us/step - loss: 1.9610 - accuracy: 0.2803
Epoch 5/200
29/29 [==============================] - 0s 606us/step - loss: 1.9367 - accuracy: 0.3424
Epoch 6/200
29/29 [==============================] - 0s 570us/step - loss: 1.8736 - accuracy: 0.3637
Epoch 7/200
29/29 [==============================] - 0s 499us/step - loss: 1.6301 - accuracy: 0.4351
Epoch 8/200
29/29 [==============================] - 0s 463us/step - loss: 1.5009 - accuracy: 0.5177
Epoch 9/200
29/29 [==============================] - 0s 499us/step - loss: 1.4074 - accuracy: 0.5776
Epoch 10/200
29/29 [==============================] - 0s 463us/step - loss: 1.2390 - accura

29/29 [==============================] - 0s 499us/step - loss: 0.0184 - accuracy: 1.0000
Epoch 161/200
29/29 [==============================] - 0s 463us/step - loss: 0.0222 - accuracy: 0.9836
Epoch 162/200
29/29 [==============================] - 0s 463us/step - loss: 0.0088 - accuracy: 1.0000
Epoch 163/200
29/29 [==============================] - 0s 463us/step - loss: 0.0322 - accuracy: 1.0000
Epoch 164/200
29/29 [==============================] - 0s 463us/step - loss: 0.0101 - accuracy: 1.0000
Epoch 165/200
29/29 [==============================] - ETA: 0s - loss: 0.0092 - accuracy: 1.00 - 0s 463us/step - loss: 0.0258 - accuracy: 0.9929
Epoch 166/200
29/29 [==============================] - 0s 463us/step - loss: 0.0362 - accuracy: 0.9760
Epoch 167/200
29/29 [==============================] - 0s 463us/step - loss: 0.0073 - accuracy: 1.0000
Epoch 168/200
29/29 [==============================] - 0s 427us/step - loss: 0.0089 - accuracy: 0.9943
Epoch 169/200
29/29 [========================

## Inference

To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class.

In [12]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [13]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, corpus)
    return res

Now we will develop a graphical user interface. Let’s use Tkinter library which is shipped with tons of useful libraries for GUI. We will take the input message from the user and then use the helper functions we have created to get the response from the bot and display it on the GUI. 

In [14]:
#Creating GUI with tkinter
import tkinter
from tkinter import *

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="blue", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Proton - your virtual assistant")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="8", height=5,
                    bd=0, bg="light blue",fg='black',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
# EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=6, y=401, height=90, width=265)
SendButton.place(x=284, y=401, height=90)

base.mainloop()

Type your question in the GUI window and click on 'Send'.